In [37]:
import spacy 
import json 
import random 

def load_data(file):
    """
    takes json file location and 
    returns python object representation of that json file 
    """
    with open(file,"r", encoding='utf-8') as f: 
        data = json.load(f) 
    return data 



def save_data(file, data): 
    with open(file,"w", encoding='utf-8') as f: 
        json.dump(data,f, indent=4)  

In [38]:
TRAIN_DATA = load_data('hp_training_data.json') 

In [4]:
def train_spacy(data, iterations): 
    """
    iterations: epochs on the training dataset 
    """
    TRAIN_DATA = data 
    nlp = spacy.blank('en')
    if "ner" not in nlp.pipe_names: 
        ner = nlp.create_pipe("ner") 
        nlp.add_pipe(ner, last=True) 




In [56]:
from tqdm.notebook import tqdm

TRAIN_DATA = load_data('hp_training_data.json') 
iterations = 30
TRAIN_DATA2 = []
for val in TRAIN_DATA:
    if val:
        TRAIN_DATA2.append(val)
nlp = spacy.blank('en')

if "ner" not in nlp.pipe_names: 
    #ner = nlp.create_pipe("ner") 
    ner = nlp.add_pipe('ner', last=True) 

for a, annotations in TRAIN_DATA2: 
    for ent in annotations.get('entities'): 
        ner.add_label(ent[2])
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
if len(other_pipes) == 0:
    optimizer = nlp.begin_training()
    for itn in tqdm(range(iterations)):
        print('starting iteration' + str(itn)) 
        random.shuffle(TRAIN_DATA2)
        losses = {}
        from spacy.training.example import Example

        for batch in spacy.util.minibatch(TRAIN_DATA2, size=50):
            for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3, sgd = optimizer)
nlp



#from spacy.training.example import Example

#for batch in spacy.util.minibatch(TRAIN_DATA2, size=2):
#    for text, annotations in batch:
#        # create Example
#        doc = nlp.make_doc(text)
#        example = Example.from_dict(doc, annotations)
#        # Update the model
#        nlp.update([example], losses=losses, drop=0.3, sgd = optimizer)

#nlp.pipe_names

  0%|          | 0/30 [00:00<?, ?it/s]

starting iteration0
starting iteration1
starting iteration2
starting iteration3
starting iteration4
starting iteration5
starting iteration6
starting iteration7
starting iteration8
starting iteration9
starting iteration10
starting iteration11
starting iteration12
starting iteration13
starting iteration14
starting iteration15
starting iteration16
starting iteration17
starting iteration18
starting iteration19
starting iteration20
starting iteration21
starting iteration22
starting iteration23
starting iteration24
starting iteration25
starting iteration26
starting iteration27
starting iteration28
starting iteration29


In [57]:
nlp.to_disk('hp_ner_model')

In [58]:
def train_spacy(data, iterations):
    from tqdm.notebook import tqdm 
    import random
    TRAIN_DATA = data 
    TRAIN_DATA2 = []
    for val  in TRAIN_DATA:
        if val:
            TRAIN_DATA2.append(val)
    
    nlp = spacy.blank('en')
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True) 
    
    for _, annotations in TRAIN_DATA2:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2]) 
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner'] 
    if len(other_pipes) == 0:
        optimizer = nlp.begin_training() 
        for itn in tqdm(range(iterations)):
            print('starting iteration:' + str(itn))
            random.shuffle(TRAIN_DATA2)
            losses = {} 
            from spacy.training.example import Example 

            for batch in spacy.util.minibatch(TRAIN_DATA2, size = 256):
                for text, annotations in batch: 
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    nlp.update([example], losses=losses, drop = 0.3, sgd = optimizer)
    return nlp 

In [60]:
test = """Hermione first met Harry Potter and Ron Weasley aboard the Hogwarts Express, who found her unfriendly and somewhat of an "insufferable know-it-all", an impression reinforced by her constant correct answers and eagerness to please the professors. However, she stepped in to take the blame from the boys after they had saved her from a troll on Hallowe'en in 1991, surprising them in a grateful way, which led to them quickly becoming friends. She later played a crucial role in protecting the Philosopher's Stone from Voldemort.

In her second year, Hermione had a key role in the discovery of the Chamber of Secrets, before falling victim to the basilisk unleashed upon Hogwarts following the opening of the Chamber. However, she recovered from the petrification under the care of Madam Pomfrey with Professor Sprout's Mandrake Restorative Draught.

The next year, Hermione was granted permission to use a Time-Turner from the Ministry of Magic to facilitate her volition to study far more subjects than were possible without time travel, though she and Harry later used it to rescue Sirius Black from the Dementor's Kiss and Buckbeak the hippogriff from execution.

During her fourth year, Hermione became an advocate for the better treatment of house-elves, forming the association S.P.E.W., and helped in Harry's preparation for the Triwizard Tournament.

In her fifth year, Hermione was the driving force behind the creation of Dumbledore's Army and fought alongside fellow D.A. members in the Battle of the Department of Mysteries."""

In [61]:
nlp = spacy.load('hp_ner_model')
doc = nlp(test)
for ent in doc.ents:
    print(ent.text, ent.label_)

Harry Potter PERSON
Ron Weasley PERSON
the Hogwarts Express ORG
1991 DATE
the Philosopher's Stone ORG
Hogwarts ORG
Chamber DATE
Madam Pomfrey PERSON
Sprout's Mandrake Restorative Draught PERSON
The next year DATE
Time-Turner ORG
the Ministry of Magic ORG
Harry later used PERSON
Sirius Black ORG
the Dementor's Kiss and Buckbeak the ORG
During PERSON
fourth year DATE
S.P.E.W. GPE
Harry PERSON
Triwizard ORG
fifth year DATE
Dumbledore's Army ORG
Mysteries GPE


In [63]:
import re 
def clean_text(text):
    cleaned = re.sub(r"[\(\[].*?[\)\]]", "", text)
    return cleaned

In [65]:
test = clean_text(test)
nlp = spacy.load('hp_ner_model')
doc = nlp(test)
for ent in doc.ents:
    print(ent.text, ent.label_)

Harry Potter PERSON
Ron Weasley PERSON
the Hogwarts Express ORG
1991 DATE
the Philosopher's Stone ORG
Hogwarts ORG
Chamber DATE
Madam Pomfrey PERSON
Sprout's Mandrake Restorative Draught PERSON
The next year DATE
Time-Turner ORG
the Ministry of Magic ORG
Harry later used PERSON
Sirius Black ORG
the Dementor's Kiss and Buckbeak the ORG
During PERSON
fourth year DATE
S.P.E.W. GPE
Harry PERSON
Triwizard ORG
fifth year DATE
Dumbledore's Army ORG
Mysteries GPE
